In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import SubsetRandomSampler
import random
import math
import copy

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [49]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 20
        self.best_lr_list = []
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
        self.gamma = 0.1
        self.alpha_max = 0.1
        self.epsilon = 8
        self.clip_threshold = 0.01
        self.split = 600
        
args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size,
    pin_memory = True

)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size,
    pin_memory = True
)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 34, 5, 1)
        self.conv2 = nn.Conv2d(34, 64, 5, 1)
        self.fc1 = nn.Linear(20*20*64, 512)
        self.fc2 = nn.Linear(512, 10)
        self.drop = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 20*20*64)
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# model is not exactully the same as the paper since it did not mention the unit of fc


In [5]:
np.random.laplace(0, 1.5/1.3, 1)

array([-0.07392263])

In [6]:
def load_grad(temp, model):
    for net1,net2 in zip(model.named_parameters(),temp.named_parameters()):
        net2[1].grad = net1[1].grad.clone()

In [28]:
def noisy_max (loss_list, p_nmax, clip_threshold):
    neg_loss_array = np.array([-x for x in loss_list])
    noise = np.random.laplace(0, clip_threshold/p_nmax, len(neg_loss_array))
    noisy_loss = neg_loss_array + noise
    best_loss_index = np.argmax(noisy_loss)
    return best_loss_index

In [17]:
def add_grad_noise(model, noise):
    for i, param in enumerate(model.parameters()):
        param.grad.add_(noise[i])

def sub_grad_noise(model, noise):
    for i, param in enumerate(model.parameters()):
        param.grad.sub_(noise[i])

In [18]:
def create_grad_Gaussian_noise(model, device, p_ng, clip_threshold, batch_size):
    noise = []
    # remembe that torch.normal(mean, std) use std 
    for param in model.parameters():
        noise.append(torch.normal(0, clip_threshold/math.sqrt(2 * p_ng), param.grad.size(), device=device)/batch_size)
    return noise


In [ ]:
model = Net()
r = np.random.randint(920)
sampler = SubsetRandomSampler(list(range(r*args.batch_size, (r+5)*args.batch_size)))
    
step_size_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    sampler=sampler,
    batch_size=args.batch_size,
    pin_memory = True
    )
for i,j in step_size_loader:
    o = model(i)
    loss = F.nll_loss(o, j)
    loss.backward()
noise = create_noise(model, device, args.epsilon, args.clip_threshold, args.batch_size)
print(noise[0])
add_grad_noise(model, noise)


In [38]:
def best_step_size_model(args, model, device, train_loader, p_ng):
    
    r = np.random.randint(920)
    sampler = SubsetRandomSampler(list(range(r*args.batch_size, (r+5)*args.batch_size)))
    
    step_size_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    sampler=sampler,
    batch_size=args.batch_size,
    pin_memory = True
    )
    
    best_loss = math.inf
    best_lr = 0
    best_model = Net().to(device)
    
    
    if not args.best_lr_list:
        args.alpha_max = min(args.alpha_max, 0.1)
    elif len(args.best_lr_list) % 10 == 0:
        args.alpha_max = (1+args.gamma) * max(args.best_lr_list)
        del args.best_lr_list[:]

    #while lr_index == 0, means choose the noise add on gradient again.    

    noise = create_grad_Gaussian_noise(model, device, p_ng, args.clip_threshold, args.batch_size)
    index = 0
    args.epsilon -= p_ng
    if args.epsilon < 0:
        return model, p_ng
    
    while index == 0:
        temp_loss_list = []
        temp_model_list = []
        temp_lr_list = []
        add_grad_noise(model, noise)
        
        for i in np.linspace(0, args.alpha_max, 21):
            temp = Net().to(device)
            temp_loss = 0
            temp.load_state_dict(model.state_dict())
            #load_state_dict will not copy the grad, so you need to copy it here.
            load_grad(temp, model)
            

            temp_optimizer = optim.SGD(temp.parameters(), lr=i)
            temp_optimizer.step()
            #optimizer will be new every time, so if you have state in optimizer, it will need load state from the old optimzer.

            for (data, target) in step_size_loader:
                data,target = data.to(device), target.to(device)
                output = model(data)
                temp_loss += F.nll_loss(output, target).item()

            temp_loss_list.append(temp_loss)
            temp_model_list.append(temp)
            temp_lr_list.append(i)
        
        #choose the best lr with noisy max
        index = noisy_max(temp_loss_list, math.sqrt(2*p_nmax), args.clip_threshold)
        args.epsilon -= p_nmax
        if args.epsilon < 0:
            return model, p_ng
        
        # if index == 0, means we need to add the noise again and cost more epsilon
        if index == 0:
            #delete the original noise and add new noise
            sub_grad_noise(model, noise)
            # create new noise, and also sub the epsilon of new noise
            p_ng = (1+args.gamma) * p_ng
            noise = create_grad_Gaussian_noise(model, device, p_ng, args.clip_threshold, args.batch_size)
            args.epsilon -= (args.gamma * p_ng)
            if args.epsilon < 0:
                break
        else :
            best_model.load_state_dict(temp_model_list[index].state_dict())
            best_loss = temp_loss_list[index]
            best_lr = temp_lr_list[index]
            
    args.best_lr_list.append(best_lr)
#     print("best learning rate:", best_lr)
#     print("best loss:", best_loss)


    return best_model, p_ng

In [39]:
def train(args, device, model, train_loader, epoch, p_ng):
    
    #init the p_nmax and p_ng
    
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        
        # Chose the best step size(learning rate)
        batch_best_model, p_ng = best_step_size_model(args, model, device, train_loader, p_ng)
        if args.epsilon < 0:
            break
        
        model.load_state_dict(batch_best_model.state_dict())
        model.zero_grad()
        #remember to zero_grad or the grad will accumlate and the model will explode
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\talpha_max: {:.6f}\tepsilon: {:.2f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader.dataset) ,
                100. * batch_idx * args.batch_size / len(train_loader.dataset), loss.item(), args.alpha_max, args.epsilon))
    return model, p_ng

In [44]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\tepsilon: {:.2f}\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / (len(test_loader.dataset)), args.epsilon))

In [50]:
#%%time
model = Net().to(device)
args.best_lr_list = []

args.alpha_max = 0.1
args.epsilon = 8
p_ng, p_nmax = args.epsilon / (2 * args.split), args.epsilon / (2 * args.split) # 30 is split number, change if we need to
epoch = 0


start = time.time()
while args.epsilon > 0:
    epoch += 1
    epoch_best_model, p_ng = train(args, device, model, train_loader , epoch, p_ng)
    model.load_state_dict(epoch_best_model.state_dict())
    test(args, device, model, test_loader)
print("Spend time:{:.1f}".format(time.time() - start))
    
if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314098	alpha_max: 0.100000	epsilon: 7.99
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.582042	alpha_max: 0.090000	epsilon: 7.85
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.753495	alpha_max: 0.076500	epsilon: 7.72
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.682755	alpha_max: 0.076500	epsilon: 7.59
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.354116	alpha_max: 0.076500	epsilon: 7.45
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.463463	alpha_max: 0.068850	epsilon: 7.32
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.309969	alpha_max: 0.068850	epsilon: 7.19
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.345216	alpha_max: 0.068850	epsilon: 7.05
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.871764	alpha_max: 0.068850	epsilon: 6.92
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.227886	alpha_max: 0.068850	epsilon: 6.79
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.208587	alpha_max: 0.061965	epsilon: 6.65
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.231822	alpha_max: 0.061965	epsilon: 6.52
Train Epoch: 1 [7